In [1]:
import pandas as pd

df = pd.read_csv("preprocessed_naver.csv")
parking_set = set(df['주차 정보'])

print(parking_set)

{'정보 없음', '주차가능\n무료', '주차가능\r\n무료', '주차가능\r\n유료', '주차 불가', '주차가능\n유료', '주차가능'}


In [16]:
import re

pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_colwidth', None)  # 긴 문자열도 생략 없이 출력
pd.set_option('display.expand_frame_repr', False)  # 가로 확장 출력

# 한국 전화번호 패턴 (지역번호/휴대폰/050/0507 포함, 공백 허용)
valid_phone_pattern = re.compile(
    r'^(02|0[3-9]{1}[0-9]{1})\s?-\s?\d{3,4}\s?-\s?\d{4}$'  # 지역번호 (02, 031 등)
    r'|^050\s?-\s?\d{3,4}\s?-\s?\d{4}$'  # 050 안심번호
    r'|^0507\s?-\s?\d{3,4}\s?-\s?\d{4}$'  # 0507 추가 ✅
    r'|^01[016789]\s?-\s?\d{3,4}\s?-\s?\d{4}$'  # 휴대폰 번호 (010, 011 등)
)

# 유효하지 않은 전화번호 찾기
invalid_phone_numbers = df[~df['전화번호'].astype(str).str.match(valid_phone_pattern, na=False)]

# 결과 출력
print(invalid_phone_numbers[['전화번호']])


               전화번호
2               NaN
13              NaN
18              NaN
22              NaN
23              NaN
35              NaN
36              NaN
38              NaN
39              NaN
45   0502-1909-2468
51              NaN
60              NaN
65              NaN
66              NaN
81              NaN
101             NaN
116             NaN
126             NaN
129             NaN
132             NaN
133             NaN
152             NaN
163             NaN
173             NaN
184             NaN
187             NaN
189             NaN
190             NaN
195             NaN
214             NaN
221             NaN
229             NaN
241             NaN
276             NaN
280             NaN
293             NaN
307             NaN
308             NaN
309             NaN
311             NaN
318             NaN
365             NaN
375  0504-3142-3531
379             NaN
381             NaN
382             NaN
397             NaN
403             NaN
404             NaN


In [21]:
import re

# 영업시간 패턴 정의 (요일 + 시간 or 요일 + 정기휴무)
operating_hours_pattern = re.compile(
    r'^(?:'
    r'(?:월|화|수|목|금|토|일):\s?\d{2}:\d{2}\s?-\s?\d{2}:\d{2};?\s?'  # 요일: HH:MM - HH:MM
    r'|(?:월|화|수|목|금|토|일):\s?정기휴무\s?\(매주\s?(월|화|수|목|금|토|일)요일\);?\s?'  # 요일: 정기휴무 (매주 OO요일)
    r')+$'
)

# 형식이 맞지 않는 데이터 찾기
invalid_operating_hours = df[~df['운영시간'].astype(str).str.match(operating_hours_pattern, na=False)]

# 결과 출력
print(invalid_operating_hours[['운영시간']])


                                                                                                                    운영시간
2                                                                                                                    NaN
6                                                                                                                    NaN
11                                                                                                                   NaN
13                                                                                                                   NaN
14                                                                                                                   NaN
16                                                                                                                   NaN
18                                                                                                                   NaN
25                              

In [23]:
# '총 리뷰 개수' 열에서 숫자가 아닌 값 찾기 (정수 및 소수 허용)
invalid_review_counts = df[~df['총 리뷰 개수'].astype(str).str.match(r'^\d+(\.\d+)?$', na=False)]

# 결과 출력
print(invalid_review_counts[['총 리뷰 개수']])


Empty DataFrame
Columns: [총 리뷰 개수]
Index: []


In [24]:
# 10글자 이하인 데이터 찾기
short_intro = df[df['소개'].astype(str).str.len() <= 10]

# 결과 출력
print(short_intro[['소개']])


            소개
2        정보 없음
11       정보 없음
13       정보 없음
18       정보 없음
28       정보 없음
34       정보 없음
79       정보 없음
89       정보 없음
96    전메뉴 포장가능
104      정보 없음
115      정보 없음
124      정보 없음
131      정보 없음
133      정보 없음
134      정보 없음
141      정보 없음
169      정보 없음
182      정보 없음
189      정보 없음
199      정보 없음
202      정보 없음
204      정보 없음
213      정보 없음
214      정보 없음
220      정보 없음
223      정보 없음
224      정보 없음
235      정보 없음
238      정보 없음
239      정보 없음
242      정보 없음
251      정보 없음
266      정보 없음
274      정보 없음
276      정보 없음
277      정보 없음
280      정보 없음
283      정보 없음
287      정보 없음
293      정보 없음
301     포장됩니다.
302      정보 없음
306      정보 없음
307      정보 없음
309      정보 없음
311      정보 없음
322      정보 없음
326      정보 없음
328      정보 없음
330      정보 없음
334      정보 없음
335      정보 없음
340      정보 없음
341      정보 없음
344      정보 없음
345      정보 없음
348      정보 없음
349      정보 없음
350      정보 없음
355      정보 없음
358      정보 없음
366      정보 없음
367      정보 없음
369      정보 없음
371      정보 없음
372      정

In [25]:
import ast

# '편의시설 및 서비스' 열을 리스트로 변환 (JSON 형식으로 저장된 경우)
df['편의시설 및 서비스'] = df['편의시설 및 서비스'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

# 비어있는 리스트([]) 찾기
empty_facilities = df[df['편의시설 및 서비스'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

# 결과 출력
print(empty_facilities[['편의시설 및 서비스']])


    편의시설 및 서비스
2           []
11          []
13          []
28          []
41          []
63          []
77          []
79          []
89          []
104         []
105         []
115         []
120         []
124         []
131         []
134         []
169         []
182         []
187         []
189         []
204         []
213         []
214         []
220         []
223         []
224         []
238         []
246         []
249         []
266         []
272         []
274         []
277         []
285         []
287         []
293         []
306         []
307         []
308         []
309         []
311         []
314         []
325         []
327         []
328         []
334         []
335         []
340         []
342         []
344         []
348         []
350         []
358         []
365         []
366         []
367         []
371         []
373         []
377         []
383         []
386         []
389         []
390         []
403         []
422         []
425       

In [ ]:
import ast

# '이런점이 좋았어요' 열을 리스트로 변환 (JSON 형식으로 저장된 경우)
df['이런점이 좋았어요'] = df['이런점이 좋았어요'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x
)

# 빈 리스트([]) 찾기
empty_good_points = df[df['이런점이 좋았어요'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

# 결과 출력
print(empty_good_points[['이런점이 좋았어요']])


    이런점이 좋았어요
293        []
503        []
516        []
564        []
604        []


In [27]:
import ast

# '좌석 정보' 열을 리스트로 변환 (JSON 형식으로 저장된 경우)
df['좌석 정보'] = df['좌석 정보'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x
)

# 빈 리스트([]) 찾기
empty_seat_info = df[df['좌석 정보'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

# 결과 출력
print(empty_seat_info[['좌석 정보']])


    좌석 정보
2      []
4      []
5      []
11     []
12     []
13     []
14     []
17     []
18     []
19     []
20     []
21     []
23     []
24     []
25     []
26     []
28     []
32     []
33     []
35     []
37     []
38     []
39     []
41     []
42     []
44     []
45     []
46     []
48     []
50     []
51     []
52     []
55     []
56     []
57     []
59     []
61     []
62     []
63     []
65     []
67     []
68     []
71     []
72     []
73     []
74     []
77     []
79     []
89     []
91     []
93     []
99     []
100    []
104    []
107    []
115    []
119    []
121    []
124    []
126    []
128    []
131    []
132    []
133    []
134    []
135    []
136    []
140    []
141    []
151    []
152    []
153    []
154    []
155    []
158    []
160    []
164    []
168    []
169    []
170    []
172    []
173    []
174    []
175    []
176    []
178    []
180    []
182    []
183    []
186    []
187    []
189    []
190    []
191    []
197    []
199    []
201    []
202    []
203    []
